In [1]:
%%capture

!pip install gensim==4.0.1

In [2]:
%%time

# Importing necessary libraries

import gensim.downloader
from gensim.models import Word2Vec
from gensim.parsing.preprocessing import preprocess_string
import json
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', None)
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

CPU times: user 939 ms, sys: 268 ms, total: 1.21 s
Wall time: 2.07 s


In [3]:
%%time


'''
Reading from input file and doing json normalizing upon the train data to achieve the required 
dataframe format
'''
def convert_from_json_to_dataframe(file_path, record_path=['data', 'paragraphs', 'qas', 'answers']):
    file = json.loads(open(file_path).read())
    js = pd.json_normalize(file, record_path)
    m = pd.json_normalize(file, record_path[:-1])
    r = pd.json_normalize(file, record_path[:-2])
    index = np.repeat(r['context'].values, r.qas.str.len())
    m['context'] = index
    data = m[['id', 'question', 'context', 'answers']].set_index('id').reset_index()
    data['context_id'] = data['context'].factorize()[0]
    return data
    

train_file_path = '../input/stanford-question-answering-dataset/train-v1.1.json'
train_data = convert_from_json_to_dataframe(train_file_path)
train_data

CPU times: user 6.26 s, sys: 338 ms, total: 6.59 s
Wall time: 7.49 s


,id,question,context,answers,context_id
0,5733be284776f41900661182,To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?,"Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend ""Venite Ad Me Omnes"". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.","[{'answer_start': 515, 'text': 'Saint Bernadette Soubirous'}]",0
1,5733be284776f4190066117f,What is in front of the Notre Dame Main Building?,"Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend ""Venite Ad Me Omnes"". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.","[{'answer_start': 188, 'text': 'a copper statue of Christ'}]",0
2,5733be284776f41900661180,The Basilica of the Sacred heart at Notre Dame is beside to which structure?,"Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend ""Venite Ad Me Omnes"". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.","[{'answer_start': 279, 'text': 'the Main Building'}]",0
3,5733be284776f41900661181,What is the Grotto at Notre Dame?,"Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend ""Venite Ad Me Omnes"". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.","[{'answer_start': 381, 'text': 'a Marian place of prayer and reflection'}]",0
4,5733be284776f4190066117e,What sits on top of the Main Building at Notre Dame?,"Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend ""Venite Ad Me Omnes"". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, Franc

In [4]:
%%time

# Exploring the train dataset a bit more!

print ('Number of unique contexts in the train dataset :', len(train_data['context_id'].unique()))

Number of unique contexts in the train dataset : 18891
CPU times: user 915 µs, sys: 2.97 ms, total: 3.88 ms
Wall time: 4.73 ms


In [5]:
%%time

# Getting unique documents from the train dataset

train_unique_documents = train_data[['context', 'context_id']].drop_duplicates().reset_index(drop=True)
train_unique_documents

CPU times: user 170 ms, sys: 4.7 ms, total: 174 ms
Wall time: 175 ms


,context,context_id
0,"Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend ""Venite Ad Me Omnes"". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.",0
1,"As at most other universities, Notre Dame's students run a number of news media outlets. The nine student-run outlets include three newspapers, both a radio and television station, and several magazines and journals. Begun as a one-page journal in September 1876, the Scholastic magazine is issued twice monthly and claims to be the oldest continuous collegiate publication in the United States. The other magazine, The Juggler, is released twice a year and focuses on student literature and artwork. The Dome yearbook is published annually. The newspapers have varying publication interests, with The Observer published daily and mainly reporting university and other news, and staffed by students from both Notre Dame and Saint Mary's College. Unlike Scholastic and The Dome, The Observer is an independent publication and does not have a faculty advisor or any editorial oversight from the University. In 1987, when some students believed that The Observer began to show a conservative bias, a liberal newspaper, Common Sense was published. Likewise, in 2003, when other students believed that the paper showed a liberal bias, the conservative paper Irish Rover went into production. Neither paper is published as often as The Observer; however, all three are distributed to all students. Finally, in Spring 2008 an undergraduate journal for political science research, Beyond Politics, made its debut.",1
2,"The university is the major seat of the Congregation of Holy Cross (albeit not its official headquarters, which are in Rome). Its main seminary, Moreau Seminary, is located on the campus across St. Joseph lake from the Main Building. Old College, the oldest building on campus and located near the shore of St. Mary lake, houses undergraduate seminarians. Retired priests and brothers reside in Fatima House (a former retreat center), Holy Cross House, as well as Columba Hall near the Grotto. The university through the Moreau Seminary has ties to theologian Frederick Buechner. While not Catholic, Buechner has praised writers from Notre Dame and Moreau Seminary created a Buechner Prize for Preaching.",2
3,"The College of Engineering was established in 1920, however, early courses in civil and mechanical engineering were a part of the College of Science since the 1870s. Today the college, housed in the Fitzpatrick, Cushing, and Stinson-Remick Halls of Engineering, includes five departments of study – aerospace and mechanical engineering, chemical and biomolecular engineering, civil engineering and geological sciences, computer science and engineering, and electrical engineering – with eight B.S. degrees offered. Additionally, the college offers five-year dual degree programs with the Colleges of Arts and Letters and of Business awarding additional B.A. and Master of Business Administration (MBA) degrees, respectively.",3
4,"All of Notre Dame's undergraduate students are a part of one of the five undergraduate colleges at the school or are in the First Year of Studies program. The First Year of Studies program was established in 1962 to guide incoming freshmen in their first year at the school before they have declared a major. Each student is given an academic advisor from the program who helps them to choose classes that give them exp

In [6]:
%%time

# Creating a corpus of tokens

train_corpus = train_unique_documents['context'].tolist()
train_corpus = [preprocess_string(t) for t in train_corpus]

CPU times: user 17 s, sys: 120 ms, total: 17.1 s
Wall time: 17.2 s


In [7]:
%%time

# Defining the Word2Vec model

vectorizer = Word2Vec(sentences=train_corpus, vector_size=300, window=5, min_count=10, workers=4, epochs=20).wv

CPU times: user 2min 4s, sys: 615 ms, total: 2min 5s
Wall time: 35.4 s


In [8]:
%%time

# Getting the 5 most similar words to a sample word

vectorizer.most_similar('tourist', topn=5)

CPU times: user 8.06 ms, sys: 2.95 ms, total: 11 ms
Wall time: 7.74 ms


[('destin', 0.7457951307296753),
 ('visitor', 0.7418142557144165),
 ('tourism', 0.6451799869537354),
 ('porto', 0.5706731677055359),
 ('windhoek', 0.521174430847168)]

In [9]:
%%time

# Text Transformation 

def text_transformation(vectorizer, text, verbose=True):
    tokens = preprocess_string(text)
    words = [vectorizer[w] for w in tokens if w in vectorizer]
    vector = [w for w in tokens if w in vectorizer]
    if verbose:
        print ('Text :', text)
        print ('Vector :', vector)
    elif len(words):
        return np.mean(words, axis=0)
    else:
        return np.zeros((300), dtype=np.float32)
    

question = 'What are the tourist spots in Spain?'    
text_transformation(vectorizer, question, True)

Text : What are the tourist spots in Spain?
Vector : ['tourist', 'spot', 'spain']
CPU times: user 7.96 ms, sys: 12 ms, total: 20 ms
Wall time: 6.34 ms


In [10]:
%%time

# Training the model

retriever_configs = {'n_neighbors' : 1, 'metric' : 'cosine'}
retriever = NearestNeighbors(**retriever_configs)
X_train = train_unique_documents['context'].apply(lambda x : text_transformation(vectorizer, x, False)).tolist()
retriever.fit(X_train, train_unique_documents['context_id'])

CPU times: user 24.8 s, sys: 45.8 ms, total: 24.9 s
Wall time: 25 s


NearestNeighbors(metric='cosine', n_neighbors=1)

In [11]:
%%time

# Evaluating the above model

X_train = train_data['question'].apply(lambda x : text_transformation(vectorizer, x, False)).tolist()

# predict one document for each question
y_test = train_data['context_id']
y_pred = retriever.kneighbors(X_train, return_distance=False)

CPU times: user 57.5 s, sys: 12.5 s, total: 1min 10s
Wall time: 38.2 s


In [12]:
%%time

# Testing the model upon the training set
def compute_accuracy(y_test, y_pred):
    num_correct, num_total = 0, len(y_test)
    for i in range(num_total):
        y_pred_list = y_pred[i]
        if y_test[i] in y_pred_list:
            num_correct += 1
    return num_correct / num_total


acc = compute_accuracy(y_test, y_pred)
print('Accuracy:', f'{acc:.4f}')

Accuracy: 0.2151
CPU times: user 728 ms, sys: 1 ms, total: 729 ms
Wall time: 731 ms


In [13]:
%%time

validation_file_path = '../input/stanford-question-answering-dataset/dev-v1.1.json'
validation_data = convert_from_json_to_dataframe(validation_file_path)
validation_data

CPU times: user 1.14 s, sys: 35.6 ms, total: 1.17 s
Wall time: 1.24 s


,id,question,context,answers,context_id
0,56be4db0acb8001400a502ec,Which NFL team represented the AFC at Super Bowl 50?,"Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi's Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the ""golden anniversary"" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as ""Super Bowl L""), so that the logo could prominently feature the Arabic numerals 50.","[{'answer_start': 177, 'text': 'Denver Broncos'}, {'answer_start': 177, 'text': 'Denver Broncos'}, {'answer_start': 177, 'text': 'Denver Broncos'}]",0
1,56be4db0acb8001400a502ed,Which NFL team represented the NFC at Super Bowl 50?,"Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi's Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the ""golden anniversary"" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as ""Super Bowl L""), so that the logo could prominently feature the Arabic numerals 50.","[{'answer_start': 249, 'text': 'Carolina Panthers'}, {'answer_start': 249, 'text': 'Carolina Panthers'}, {'answer_start': 249, 'text': 'Carolina Panthers'}]",0
2,56be4db0acb8001400a502ee,Where did Super Bowl 50 take place?,"Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi's Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the ""golden anniversary"" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as ""Super Bowl L""), so that the logo could prominently feature the Arabic numerals 50.","[{'answer_start': 403, 'text': 'Santa Clara, California'}, {'answer_start': 355, 'text': 'Levi's Stadium'}, {'answer_start': 355, 'text': 'Levi's Stadium in the San Francisco Bay Area at Santa Clara, California.'}]",0
3,56be4db0acb8001400a502ef,Which NFL team won Super Bowl 50?,"Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi's Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the ""golden anniversary"" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as ""Super Bowl L""), so that the logo could prominently feature the Arabic numerals 50.","[{'answer_start

In [14]:
%%time

# Exploring the validation dataset a bit more!

print ('Number of unique contexts in the validation dataset :', len(validation_data['context_id'].unique()))

Number of unique contexts in the validation dataset : 2067
CPU times: user 899 µs, sys: 0 ns, total: 899 µs
Wall time: 862 µs


In [15]:
%%time

# Getting unique documents from the validation dataset

validation_unique_documents = validation_data[['context', 'context_id']].drop_duplicates().reset_index(drop=True)
validation_unique_documents

CPU times: user 26.8 ms, sys: 982 µs, total: 27.7 ms
Wall time: 27.7 ms


,context,context_id
0,"Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi's Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the ""golden anniversary"" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as ""Super Bowl L""), so that the logo could prominently feature the Arabic numerals 50.",0
1,"The Panthers finished the regular season with a 15–1 record, and quarterback Cam Newton was named the NFL Most Valuable Player (MVP). They defeated the Arizona Cardinals 49–15 in the NFC Championship Game and advanced to their second Super Bowl appearance since the franchise was founded in 1995. The Broncos finished the regular season with a 12–4 record, and denied the New England Patriots a chance to defend their title from Super Bowl XLIX by defeating them 20–18 in the AFC Championship Game. They joined the Patriots, Dallas Cowboys, and Pittsburgh Steelers as one of four teams that have made eight appearances in the Super Bowl.",1
2,"The Broncos took an early lead in Super Bowl 50 and never trailed. Newton was limited by Denver's defense, which sacked him seven times and forced him into three turnovers, including a fumble which they recovered for a touchdown. Denver linebacker Von Miller was named Super Bowl MVP, recording five solo tackles, 2½ sacks, and two forced fumbles.",2
3,"CBS broadcast Super Bowl 50 in the U.S., and charged an average of $5 million for a 30-second commercial during the game. The Super Bowl 50 halftime show was headlined by the British rock group Coldplay with special guest performers Beyoncé and Bruno Mars, who headlined the Super Bowl XLVII and Super Bowl XLVIII halftime shows, respectively. It was the third-most watched U.S. broadcast ever.",3
4,"In early 2012, NFL Commissioner Roger Goodell stated that the league planned to make the 50th Super Bowl ""spectacular"" and that it would be ""an important game for us as a league"".",4
...,...,...
2062,"where is the mass of the object, is the velocity of the object and is the distance to the center of the circular path and is the unit vector pointing in the radial direction outwards from the center. This means that the unbalanced centripetal force felt by any object is always directed toward the center of the curving path. Such forces act perpendicular to the velocity vector associated with the motion of an object, and therefore do not change the speed of the object (magnitude of the velocity), but only the direction of the velocity vector. The unbalanced force that accelerates an object can be resolved into a component that is perpendicular to the path, and one that is tangential to the path. This yields both the tangential force, which accelerates the object by either slowing it down or speeding it up, and the radial (centripetal) force, which changes its direction.",2062
2063,"A conservative force that acts on a closed system has an associated mechanical work that allows energy to convert only between kinetic or potential forms. This means that for a closed system, the net mechanical energy is conserved whenever a conservative force acts on the system. The force, therefore, is related directly to the difference in potential energy between two different locations in space, and can be considered to be an artifact of the potential field in the same way that the direction and amount of a flow of water can be considered to be an artifact of the contour map of the elevation of an area.",2063
2064,"For certain physical scenarios, it is impossible to model forces a

In [16]:
%%time

# Creating a corpus of tokens

validation_corpus = validation_unique_documents['context'].tolist()
validation_corpus = [preprocess_string(t) for t in validation_corpus]

CPU times: user 2.17 s, sys: 21.5 ms, total: 2.19 s
Wall time: 2.2 s


In [17]:
%%time

# Defining the Word2Vec model

vectorizer = Word2Vec(sentences=validation_corpus, vector_size=300, window=5, min_count=10, workers=4, epochs=20).wv

CPU times: user 10.8 s, sys: 85.2 ms, total: 10.9 s
Wall time: 3.43 s


In [18]:
%%time

# Getting the 5 most similar words to a sample word

vectorizer.most_similar('tourist', topn=5)

CPU times: user 2.18 ms, sys: 1.86 ms, total: 4.05 ms
Wall time: 2.26 ms


[('southeast', 0.9052872657775879),
 ('tradition', 0.8859097361564636),
 ('denmark', 0.8834764361381531),
 ('commonwealth', 0.8831672668457031),
 ('destin', 0.8796497583389282)]

In [19]:
%%time

# Text Transformation

question = 'What are the tourist spots in Spain?'    
text_transformation(vectorizer, question, True)

Text : What are the tourist spots in Spain?
Vector : ['tourist', 'spain']
CPU times: user 4.22 ms, sys: 1.89 ms, total: 6.12 ms
Wall time: 1.56 ms


In [20]:
%%time

# Training the model

retriever_configs = {'n_neighbors' : 1, 'metric' : 'cosine'}
retriever = NearestNeighbors(**retriever_configs)
X_validation = validation_unique_documents['context'].apply(lambda x : text_transformation(vectorizer, x, False)).tolist()
retriever.fit(X_validation, validation_unique_documents['context_id'])

CPU times: user 2.8 s, sys: 4.01 ms, total: 2.81 s
Wall time: 2.82 s


NearestNeighbors(metric='cosine', n_neighbors=1)

In [21]:
%%time

# Evaluating the above model

X_validation = validation_data['question'].apply(lambda x : text_transformation(vectorizer, x, False)).tolist()

# predict one document for each question
y_test = validation_data['context_id']
y_pred = retriever.kneighbors(X_validation, return_distance=False)

CPU times: user 2.41 s, sys: 362 ms, total: 2.78 s
Wall time: 2.05 s


In [22]:
%%time

# Testing the model upon the validation set

acc = compute_accuracy(y_test, y_pred)
print('Accuracy:', f'{acc:.4f}')

Accuracy: 0.1376
CPU times: user 91.6 ms, sys: 5 µs, total: 91.6 ms
Wall time: 92 ms
